In [1]:
import numpy as np
import pandas as pd
import os

pd.options.display.max_colwidth = 1000

In [2]:
csv_save_dir = 'output_csvs'
if not os.path.exists(csv_save_dir):
    os.makedirs(csv_save_dir)

In [3]:
%%time

# Read information for the new records
records = pd.read_csv('new-citations-rerun.tsv', sep='\t', header=None)
records.columns = ['recID', 'oldRef', 'newRef', 'metaData']

print('Total Records: {}'.format(len(records)))

Total Records: 30230936
CPU times: user 2min 39s, sys: 8.49 s, total: 2min 47s
Wall time: 2min 50s


In [4]:
# Read the new citation counts
labs = pd.read_csv('new-citation-counts-rerun.tsv', sep='\t', header=None)
labs.columns = ['recID', 'Citation_Difference']

# Index by recIDs
labs_indexed = labs.set_index('recID')

In [27]:
# Read the citation counts before update
labs_old = pd.read_csv('new-citation-counts.tsv', sep='\t', header=None)
labs_old.columns = ['recID', 'Citation_Difference']

# Index by recIDs
labs_indexed_old = labs_old.set_index('recID')

In [28]:
# Read the legacy citation counts
legacy = pd.read_csv('old-citation-counts.tsv', sep='\t')
legacy.columns = ['recID', 'Citation_Difference']

# Index by recrod IDs
legacy_indexed = legacy.set_index('recID')

In [29]:
# Compute the difference in citation counts between Labs and Legacy
diff = labs_indexed - legacy_indexed
diff_old = labs_indexed_old - legacy_indexed

In [30]:
# Filter out the NaNs
is_nan = diff.Citation_Difference.isna()
NaN_entries = diff[is_nan]

print('{} articles are differently indexed between the legacy and labs records and have no correspondence'.format(len(NaN_entries)))

is_nan_old = diff_old.Citation_Difference.isna()
NaN_entries_old = diff_old[is_nan_old]

print('{} articles are differently indexed between the legacy and labs (old run) records and have no correspondence'.format(len(NaN_entries_old)))

11434 articles are differently indexed between the legacy and labs records and have no correspondence
6032 articles are differently indexed between the legacy and labs (old run) records and have no correspondence


In [31]:
# Filter the valid entries
is_valid = ~diff.Citation_Difference.isna()
valid_diff = diff[is_valid]

is_valid_old = ~diff_old.Citation_Difference.isna()
valid_diff_old = diff_old[is_valid_old]

In [32]:
# Get the articles which have lost citations
lost = valid_diff.Citation_Difference < 0
lost_citations = valid_diff[lost]
print('Total {} articles lost citations in the Labs algorithm, with maximum lost count being {}'.format(
    len(lost_citations), lost_citations.min()))

# Sort the list
lost_citations = lost_citations.sort_values('Citation_Difference')

Total 50426 articles lost citations in the Labs algorithm, with maximum lost count being Citation_Difference   -1148.0
dtype: float64


In [33]:
# Get the articles which have lost (old) citations
lost_old = valid_diff_old.Citation_Difference < 0
lost_citations_old = valid_diff_old[lost_old]
print('Total {} articles lost citations in the old Labs algorithm, with maximum lost count being {}'.format(
    len(lost_citations_old), lost_citations_old.min()))

# Sort the list
lost_citations_old = lost_citations_old.sort_values('Citation_Difference')

Total 46256 articles lost citations in the old Labs algorithm, with maximum lost count being Citation_Difference   -1023.0
dtype: float64


In [34]:
# Get the articles which have gained citations
gained = valid_diff.Citation_Difference > 0
gained_citations = valid_diff[gained]
print('Total {} articles gained citations in the Labs algorithm, with maximum gained count being {}'.format(
    len(gained_citations), gained_citations.max()))

# Sort the list
gained_citations = gained_citations.sort_values('Citation_Difference', ascending=False)

Total 17141 articles gained citations in the Labs algorithm, with maximum gained count being Citation_Difference    267.0
dtype: float64


In [35]:
# Get the articles which have gained (old) citations
gained_old = valid_diff_old.Citation_Difference > 0
gained_citations_old = valid_diff_old[gained_old]
print('Total {} articles gained citations in the old Labs algorithm, with maximum gained count being {}'.format(
    len(gained_citations_old), gained_citations_old.max()))

# Sort the list
gained_citations_old = gained_citations_old.sort_values('Citation_Difference', ascending=False)

Total 19416 articles gained citations in the old Labs algorithm, with maximum gained count being Citation_Difference    408.0
dtype: float64


In [36]:
# Get the counts for absolute differences in citation numbers
abs_diff = np.abs(valid_diff)
abs_diff = abs_diff.sort_values('Citation_Difference', ascending=False)

# Filter out the zero difference entries
non_zero_diff = abs_diff.Citation_Difference != 0
abs_diff = abs_diff[non_zero_diff]

In [37]:
# Get the counts for absolute differences in citation numbers (old)
abs_diff_old = np.abs(valid_diff_old)
abs_diff_old = abs_diff_old.sort_values('Citation_Difference', ascending=False)

# Filter out the zero difference entries
non_zero_diff_old = abs_diff_old.Citation_Difference != 0
abs_diff_old = abs_diff_old[non_zero_diff_old]

In [38]:
# Initialize extended DataFrames for the differences
lost_citations_ext = lost_citations
gained_citations_ext = gained_citations
abs_diff_ext = abs_diff

# For lost citations
lost_citations_ext['Legacy_Count'] = legacy_indexed.loc[lost_citations.index]
lost_citations_ext['Labs_Count'] = labs_indexed.loc[lost_citations.index]

# For gained citations
gained_citations_ext['Legacy_Count'] = legacy_indexed.loc[gained_citations.index]
gained_citations_ext['Labs_Count'] = labs_indexed.loc[gained_citations.index]

# For absolute difference in citation counts
abs_diff_ext['Legacy_Count'] = legacy_indexed.loc[abs_diff.index]
abs_diff_ext['Labs_Count'] = labs_indexed.loc[abs_diff.index]

In [40]:
# Initialize extended DataFrames for the differences (old)
lost_citations_ext_old = lost_citations_old
gained_citations_ext_old = gained_citations_old
abs_diff_ext_old = abs_diff_old

# For lost citations
lost_citations_ext_old['Legacy_Count'] = legacy_indexed.loc[lost_citations_old.index]
lost_citations_ext_old['Labs_Count'] = labs_indexed_old.loc[lost_citations_old.index]

# For gained citations
gained_citations_ext_old['Legacy_Count'] = legacy_indexed.loc[gained_citations_old.index]
gained_citations_ext_old['Labs_Count'] = labs_indexed_old.loc[gained_citations_old.index]

# For absolute difference in citation counts
abs_diff_ext_old['Legacy_Count'] = legacy_indexed.loc[abs_diff_old.index]
abs_diff_ext_old['Labs_Count'] = labs_indexed_old.loc[abs_diff_old.index]

### Some useful statistics

In [41]:
# Indicate Number of articles with zero differences in citation counts
num_cited_legacy = len(legacy.index.unique())
num_cited_labs = len(labs.index.unique())
num_unique_articles = len(records.recID.unique())
print('Total number of unique Cited articles as per legacy: {}'.format(num_cited_legacy))
print('Total number of unique Cited articles as per Labs: {}'.format(num_cited_labs))
print('Difference in Number between the Legacy and Labs Records: {}'.format(num_cited_legacy - num_cited_labs))
print('\nTotal Number of Unique Articles which cite other papers: {}\n'.format(num_unique_articles))

num_zero_diff = valid_diff.Citation_Difference.value_counts()[0]
zero_diff = num_zero_diff / num_cited_legacy * 100
print('Number of Cited articles with zero difference: {}'.format(num_zero_diff))
print('Percentage of Cited articles with zero difference: {0:0.2f} %'.format(zero_diff))

Total number of unique Cited articles as per legacy: 760203
Total number of unique Cited articles as per Labs: 753443
Difference in Number between the Legacy and Labs Records: 6760

Total Number of Unique Articles which cite other papers: 942172

Number of Cited articles with zero difference: 683539
Percentage of Cited articles with zero difference: 89.92 %


In [42]:
# Indicate Number of articles with zero differences in citation counts (old)
num_cited_legacy_old = len(legacy.index.unique())
num_cited_labs_old = len(labs_old.index.unique())
num_unique_articles_old = len(records.recID.unique())
print('Total number of unique Cited articles as per legacy: {}'.format(num_cited_legacy_old))
print('Total number of unique Cited articles as per Labs: {}'.format(num_cited_labs_old))
print('Difference in Number between the Legacy and Labs Records: {}'.format(num_cited_legacy_old - num_cited_labs_old))
print('\nTotal Number of Unique Articles which cite other papers: {}\n'.format(num_unique_articles_old))

num_zero_diff_old = valid_diff_old.Citation_Difference.value_counts()[0]
zero_diff_old = num_zero_diff_old / num_cited_legacy_old * 100
print('Number of Cited articles with zero difference: {}'.format(num_zero_diff_old))
print('Percentage of Cited articles with zero difference: {0:0.2f} %'.format(zero_diff_old))

Total number of unique Cited articles as per legacy: 760203
Total number of unique Cited articles as per Labs: 759481
Difference in Number between the Legacy and Labs Records: 722

Total Number of Unique Articles which cite other papers: 942172

Number of Cited articles with zero difference: 691154
Percentage of Cited articles with zero difference: 90.92 %


In [43]:
# Percentage Differences for absolute differences
abs_diff_counts = abs_diff.Citation_Difference.value_counts()
print('Percent (%) of Citation Counts Difference in Labs')
print(abs_diff_counts[:5] / num_cited_legacy * 100)

num_grt5 = sum(abs_diff_counts[6:])
percent_grt5 = num_grt5 / num_cited_legacy * 100
print('\nTotal {} records with difference greater than 5 - Percentage: {} %\n'.format(num_grt5, percent_grt5))

Percent (%) of Citation Counts Difference in Labs
1.0    7.017994
2.0    1.024989
3.0    0.321493
4.0    0.149960
5.0    0.084451
Name: Citation_Difference, dtype: float64

Total 2198 records with difference greater than 5 - Percentage: 0.28913329729032905 %



In [44]:
# Percentage Differences for absolute differences (old)
abs_diff_counts_old = abs_diff_old.Citation_Difference.value_counts()
print('Percent (%) of Citation Counts Difference in Labs')
print(abs_diff_counts_old[:5] / num_cited_legacy_old * 100)

num_grt5_old = sum(abs_diff_counts_old[6:])
percent_grt5_old = num_grt5_old / num_cited_legacy_old * 100
print('\nTotal {} records with difference greater than 5 - Percentage: {} %\n'.format(num_grt5_old, percent_grt5_old))

Percent (%) of Citation Counts Difference in Labs
1.0    6.652828
2.0    0.992498
3.0    0.330307
4.0    0.168508
5.0    0.100499
Name: Citation_Difference, dtype: float64

Total 2996 records with difference greater than 5 - Percentage: 0.3941052587269453 %



In [46]:
# Percentage Differences for lost citations
lost_citations_count = lost_citations.Citation_Difference.value_counts()
lost_percent_diff = lost_citations_count/ num_cited_legacy * 100
print('Percent (%) of Citation Counts Lost in Labs')
print(lost_percent_diff[:-5])

lost_grt5 = sum(lost_citations_count[-6:])
lost_percent_grt5 = lost_grt5 / num_cited_legacy * 100
print('\nTotal {} records with lost citations greater than 5 - Percentage: {} %\n'.format(lost_grt5, lost_percent_grt5))

Percent (%) of Citation Counts Lost in Labs
-1.0    5.254912
-2.0    0.782160
-3.0    0.227439
-4.0    0.101552
-5.0    0.055906
Name: Citation_Difference, dtype: float64

Total 1606 records with lost citations greater than 5 - Percentage: 0.21125936098647335 %



In [45]:
# Percentage Differences for lost citations (old)
lost_citations_count_old = lost_citations_old.Citation_Difference.value_counts()
lost_percent_diff_old = lost_citations_count_old/ num_cited_legacy_old * 100
print('Percent (%) of Citation Counts Lost in Labs')
print(lost_percent_diff_old[:-5])

lost_grt5_old = sum(lost_citations_count_old[-6:])
lost_percent_grt5_old = lost_grt5_old / num_cited_legacy_old * 100
print('\nTotal {} records with lost citations greater than 5 - Percentage: {} %\n'.format(lost_grt5_old, lost_percent_grt5_old))

Percent (%) of Citation Counts Lost in Labs
-1.0    4.818450
-2.0    0.701655
-3.0    0.201788
-4.0    0.091949
-5.0    0.050381
Name: Citation_Difference, dtype: float64

Total 1676 records with lost citations greater than 5 - Percentage: 0.22046742777915898 %



In [47]:
# Percentage Differences for gained citations
gained_citations_count = gained_citations.Citation_Difference.value_counts()
gained_percent_diff = gained_citations_count/ num_cited_legacy * 100
print('Percent (%) of Citation Counts Gained in Labs')
print(gained_percent_diff[:5])

gained_grt5 = sum(gained_citations_count[6:])
gained_percent_grt5 = gained_grt5 / num_cited_legacy * 100
print('\nTotal {} records with gained citations greater than 5 - Percentage: {} %\n'.format(gained_grt5, gained_percent_grt5))

Percent (%) of Citation Counts Gained in Labs
1.0    1.763082
2.0    0.242830
3.0    0.094054
4.0    0.048408
5.0    0.028545
Name: Citation_Difference, dtype: float64

Total 592 records with gained citations greater than 5 - Percentage: 0.07787393630385568 %



In [48]:
# Percentage Differences for gained citations (old)
gained_citations_count_old = gained_citations_old.Citation_Difference.value_counts()
gained_percent_diff_old = gained_citations_count_old/ num_cited_legacy_old * 100
print('Percent (%) of Citation Counts Gained in Labs')
print(gained_percent_diff_old[:5])

gained_grt5_old = sum(gained_citations_count_old[6:])
gained_percent_grt5_old = gained_grt5_old / num_cited_legacy_old * 100
print('\nTotal {} records with gained citations greater than 5 - Percentage: {} %\n'.format(gained_grt5_old, gained_percent_grt5_old))

Percent (%) of Citation Counts Gained in Labs
1.0    1.834378
2.0    0.290843
3.0    0.128518
4.0    0.076558
5.0    0.050118
Name: Citation_Difference, dtype: float64

Total 1320 records with gained citations greater than 5 - Percentage: 0.17363783094778631 %



### Further checking functionality

In [ ]:
# Get all the records in both oldRef and newRef which are unique
old_unique = pd.DataFrame(records.oldRef.unique())
new_unique = pd.DataFrame(records.newRef.unique())

In [ ]:
# Get the lists of records which are in Labs but not Legacy and vice-versa
old_unique_diff = np.isin(old_unique, new_unique, assume_unique=True)
new_unique_diff = np.isin(new_unique, old_unique, assume_unique=True)

labs_only = new_unique[~new_unique_diff]
legacy_only = old_unique[~old_unique_diff]

In [ ]:
# Import the records as CSV
legacy_only.to_csv(os.path.join(csv_save_dir, 'legacy_only.csv'), index=False)
labs_only.to_csv(os.path.join(csv_save_dir, 'labs_only.csv'), index=False)

### Queries for most citation differences

In [76]:
gained_citations

,Citation_Difference,Legacy_Count,Labs_Count
recID,,,
299778,1148.0,1405,257
50008,1100.0,1577,477
181166,901.0,1644,743
1390184,856.0,863,7
87997,713.0,846,133
54961,571.0,599,28
279039,470.0,772,302
673262,390.0,438,48
9113,344.0,1414,1070


In [80]:
len(records[records.newRef == 83610])

54

In [72]:
records[(records.oldRef != records.newRef) & (records.oldRef!=0) & (records.newRef==0)]

,recID,oldRef,newRef,metaData
130,409403,51188,0,"{'reference': {'authors': [{'full_name': 'Glashow, S.L.'}], 'imprint': {'publisher': 'IOP'}, 'label': '2', 'publication_info': {'journal_title': 'Nucl.Phys.B', 'artid': '579', 'year': '1961', 'page_start': '579', 'journal_volume': '22'}, 'misc': ['S. Weinberg A. Salam N. Svartholm (eds.), Elementary Particle Theory S. L. Glashow, Almqvist and Wiksell, Stockholm (1968) 367 J. Il', 'Additional pubnote: Phys.Rev.Lett.,19,1264']}, 'record': {'$ref': 'http://localhost:5000/api/literature/51188'}, 'recid': 51188, 'curated_relation': False}"
245,409403,888357,0,"{'reference': {'authors': [{'full_name': 'Martin, E.'}], 'report_numbers': ['ALEPH-94-157'], 'label': '82', 'misc': [""Measurement of heavy avours leptons'b purity, ALEPH internal report (1994)""]}, 'record': {'$ref': 'http://localhost:5000/api/literature/888357'}, 'recid': 888357, 'curated_relation': False}"
397,407835,303781,0,"{'reference': {'publication_info': {'journal_title': 'Nucl.Instrum.Meth.A', 'artid': '1', 'year': '1990', 'page_start': '1', 'journal_volume': '288'}, 'label': '9', 'authors': [{'full_name': 'Fretwurst, E.'}, {'full_name': 'Herdan, H.'}, {'full_name': 'Lindström, G.'}, {'full_name': 'Pein, U.'}, {'full_name': 'Rollwagen, M.'}, {'full_name': 'Schatz, H.'}, {'full_name': 'Thomsen, P.'}, {'full_name': 'Wunstorf, R.'}]}, 'record': {'$ref': 'http://localhost:5000/api/literature/303781'}, 'recid': 303781, 'curated_relation': False}"
1516,407751,247422,0,"{'reference': {'publication_info': {'journal_title': 'Conf.Proc.C', 'artid': '28', 'page_start': '28', 'journal_volume': '870416'}, 'label': '1', 'authors': [{'full_name': 'Prokoshkin'}, {'full_name': 'Yu.D.'}], 'misc': [""Proc. II Int. Conf. on Hadron Spectroscopy HADRON'87, KEK, Tsukuba 1987, pp. 28-39""]}, 'legacy_curated': True, 'record': {'$ref': 'http://localhost:5000/api/literature/247422'}, 'recid': 247422, 'curated_relation': True}"
1517,407751,257073,0,"{'reference': {'publication_info': {'journal_title': 'Conf.Proc.C', 'artid': '19', 'page_start': '19', 'journal_volume': '870416'}, 'label': '2', 'authors': [{'full_name': 'Binon, F.'}], 'misc': [""Proc. II Int. Conf. on Hadron Spectroscopy HADRON'87, KEK, Tsukuba 1987, pp. 19-27""]}, 'legacy_curated': True, 'record': {'$ref': 'http://localhost:5000/api/literature/257073'}, 'recid': 257073, 'curated_relation': True}"
1522,407751,291941,0,"{'reference': {'publication_info': {'journal_title': ""Proc.III Int.Conf. on Hadron Spectroscopy HADRON'89"", 'artid': '1989', 'page_start': '1989', 'journal_volume': 'Ajaccio'}, 'label': '7', 'authors': [{'full_name': 'Prokoshkin'}, {'full_name': 'Yu.D.'}], 'misc': ['Additional pubnote: Conf.Proc.,C890923,27']}, 'legacy_curated': True, 'record': {'$ref': 'http://localhost:5000/api/literature/291941'}, 'recid': 291941, 'curated_relation': True}"
1560,409037,395678,0,"{'reference': {'title': {'title': 'Particle physics'}, 'publication_info': {'year': '1995'}, 'label': '21', 'authors': [{'full_name': 'M.Yu'}], 'misc': ['Khlopov, A.S Sakharov, A.A. Shkiyaev and A.L, Sudarikov, Proc. 1 Int. Conf. on Cosmo M.Yu (eds.) Khlopov, et al.,. To be published by Editions Frontieres']}, 'record': {'$ref': 'http://localhost:5000/api/literature/395678'}, 'recid': 395678, 'curated_relation': False}"
1566,409037,251393,0,"{'reference': {'publication_info': {'journal_title': 'Phys.Lett.B', 'artid': '525', 'page_start': '525', 'journal_volume': '103'}, 'label': '24', 'authors': [{'full_name': 'Ellis, J.'}, {'full_name': 'Olive, K.A.'}], 'misc': ['G. RaKelt and D. Seckel', 'Additional pubnote: Phys.Rev.Lett.,60,1793']}, 'record': {'$ref': 'http://localhost:5000/api/literature/251393'}, 'recid': 251393, 'curated_relation': False}"
1590,409037,14801,0,"{'reference': {'label': '33'}, 'legacy_curated': True, 'record': {'$ref': 'http://localhost:5000/api/literature/14801'}, 'recid': 14801, 'curated_relation': True}"
1637,409228,9113,0,"{'reference': {'authors': [{'full_name': 'Euler, A.'},